In [2]:
from load_data import *

In [6]:

def load_data(filename):
    df = pd.read_csv(filename)
    df = df.loc[:,["id","name","price"]]
    df.to_csv("search_embed.csv", index=False)
    load_s3("search_embed.csv")

    df["price"] = df["price"].apply(lambda x:round(Decimal(x),2))
    
    # save to csv file in s3 bucket
    search_names = df["name"].tolist()
    search_embed = model.encode(search_names, convert_to_tensor=True)
    arr = search_embed.cpu().numpy()
    byte_list = [vec.tobytes() for vec in arr]
    df["encoded"] = byte_list
    # save to dynamodb
    df_data = df.to_dict(orient="records")
    logger.debug(df_data)
    table = __get_table("price-info")

    with table.batch_writer() as writer:
        for item in df_data:
            writer.put_item(Item=item)
    logger.info("Data loaded")
    return arr


In [7]:
load_data("data/electronics_products.csv")

\x11=[\xb6\xcd=\xd43\x05=\x17\xfcf\xbd\xf0\x89\x18\xbe\xdc\xa8\x86>\x83E6>\xc8\x9b\xa1=ge\x07\xbc0k\x0b>\x08\x96\xbc\xbc\x0b\xf8j\xbd\xf2_\x9b<\x1b\x052\xbeO\t>>\xbe\xa7\x89=,\xc3\xd2=\xd7"\x0e=E1:\xbeK\xc1\xf8\xbd\xc9\xe2\xa1\xbb>\x0e\x97<\x8e\xdc\x86\xba\xf9\x90\x96;\xcc\xe4\x8f=\xd4\x95\x16>\x1fv\xc4=\x07XZ>\xf9\xbe\x16\xbe\xa5\xe6\xbb=.:\xde\xba=\xa0\xaf\xbdD\xdc\xe5=\xf2!\xfc\xbb\x89\xc4\xfd=\x05\xa3\x87=\xbc%\xae\xbd\x99t\xf7<\xf4?\x19>\x14n\x8c<\x1drP<$\x0f\xae\xbd\x94\xc9\x94\xbd\x12\xe6\x88\xbd7[\xab\xbd\xcc\x92\x95=\xd9\x07\xca\xbda\xef\x1c=R\x16->\x86\x87^\xbd\xc7\xf8\xed\xbb9\x9c\xb7\xbcI\x81A=\xc6k\x83=K\xc2\x1b>\x98\xdb\xc2=\x0b\xc7\xc8=\x01\xe5\x1a<L!\x1d>\x95\xcb\r=\xed\x0829\xab\x1e\xf7\xbb\xb9\xaa\x03\xbd\xba\xecH\xbc\x02 \xb1;\x8e\x19\x89\xbd%\x9d\x1c\xbdi\xcc\xa3\xbd\x0cqx\xbd\x16#\x8d\xbc\xc0\xfe\xdd\xbd\xc2\xe5\x10\xbd\xe2\xcf\xb8=\xf8y\x88\xbd\x14A1\xbe\xea\xf9\x13>\xb9\x82\xbf\xbc\x85\xd0\x90=\x8e\x1fm\xba\xfb\x8c\xb4>\x15%\x81<$\xed\\\xbd\x1bS-\xbe\xf75\x1b>[{\

In [5]:
def __get_table(table_name):

    db = boto3.resource('dynamodb', region_name=REGION_NAME)
    return db.Table(table_name)